In [1]:
from importlib import reload
import numpy as np

In [2]:
%cd /content/drive/MyDrive/Colab_Scratch/steps
import step
# from step import *

/content/drive/MyDrive/Colab_Scratch/steps


In [104]:
%cd /content/drive/MyDrive/Colab_Scratch/steps
reload(step)
from step import *

/content/drive/MyDrive/Colab_Scratch/steps


# [STEP 01] 상자로서의 변수

##1.2 Variable 클래스 구현

In [4]:
data = np.array(1.)
x = Variable(data)
x.data

array(1.)

In [5]:
x.data = np.array(2.)
x.data

array(2.)

# [STEP 02] 변수를 낳는 함수

##2.3 Function 클래스 이용

In [6]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2 # 구체적인 계산은 forward 메서드에서 한다
        y = as_array(y)
        output = Variable(y)
        return output

In [7]:
x = Variable(np.array(10.))
f = Function()

In [8]:
y = f(x)
print(type(y))

<class 'step.Variable'>


In [9]:
print(y.data)

100.0


In [10]:
f = Square()
y = f(x)

In [11]:
print(type(y))
print(y.data)

<class 'step.Variable'>
100.0


#[STEP 03] 함수 연결

In [12]:
f = Exp()

In [13]:
y = f(x)
y.data

array(22026.46579481)

In [14]:
x = Variable(np.array(0.5))
A = Square()
B = Exp()
C = Square()

In [15]:
a = A(x)
b = B(a)
c = C(b)
c.data

array(1.64872127)

# [STEP 04] 수치 미분

##4.2 수치 미분 구현

In [16]:
f = Square()
x = Variable(np.array(2.))
print(type(x))
# numerical_diff(f, x)

<class 'step.Variable'>


##4.3 합성 함수의 미분

In [17]:
def f(x):
    A = Square()
    B = Exp()   
    C = Square()    
    return C(B(A(x)))

In [18]:
x = Variable(np.array(0.5))
# numerical_diff(f, x)

# [STEP 05] 역전파 이론

# [STEP 06] 수동 역전파

## 6.4 역전파 구현

###순전파

In [19]:
A = Square()
B = Exp()
C = Square()

In [20]:
input = .5
x = Variable(np.array(input))

In [21]:
a = A(x)
b = B(a)
y = C(b)

In [22]:
y.data

array(1.64872127)

###역전파

In [23]:
y.grad = np.array(1.)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

In [24]:
x.grad

3.297442541400256

In [25]:
def f(input):
    A = Square()
    B = Exp()
    C = Square()

    a = A(input)
    b = B(a)
    y = C(b)
    return y

In [26]:
# numerical_diff(f, Variable(input))

# [STEP 07] 역전파 자동화

In [27]:
A = Square()
B = Exp()
C = Square()
x = Variable(np.array(.5))

In [28]:
a = A(x)
b = B(a)
y = C(b)

In [29]:
assert a.creator.input == x

##7.2 역전파 도전!

In [30]:
y.grad = np.array(1.)

In [31]:
C = y.creator

In [32]:
b = C.input

In [33]:
b.grad = C.backward(y.grad)

In [34]:
b.grad

2.568050833375483

In [35]:
B = b.creator

In [36]:
a = B.input

In [37]:
a.grad = B.backward(b.grad)

In [38]:
a.grad

3.297442541400256

In [39]:
a.creator.input.grad = a.creator.backward(a.grad)

In [40]:
x.grad

3.297442541400256

In [41]:
a
a.grad
F = a.creator
a.creator.backward
a.creator.input
a.creator.input.grad

3.297442541400256

## 7.3 backward 메서드 추가

In [42]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()

In [43]:
x.grad

3.297442541400256

# [STEP 08] 재귀에서 반복문으로

## 8.3 동작확인

In [44]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# [STEP09] 함수를 더 편리하게

##9.1 파이썬 함수로 이용하기

In [45]:
x = Variable(np.array(.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.)
y.backward()
x.grad

3.297442541400256

##9.2 bakcward 메서드 간소화

In [46]:
x = Variable(np.array(.5))
y = square(exp(square(x)))
y.backward()
x.grad

3.297442541400256

#[STEP10] 테스트

##10.1 파이썬 단위 테스트

In [87]:
%cd /content/drive/MyDrive/Colab_Scratch

/content/drive/MyDrive/Colab_Scratch


In [65]:
!ls

main.ipynb  steps  tests


In [88]:
# test_forward
!python -m unittest steps/step.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


##10.2 square 함수의 역전파 테스트

In [89]:
# test_backward
!python -m unittest steps/step.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


##10.3 기울기 확인을 이용한 자동 테스트

In [90]:
# test_gradient_check
!python -m unittest steps/step.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


# [STEP11] 가변 길이 인수(순전파 편)

##11.2 Add 클래스 구현

In [105]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
y.data

array(5)